# タイタニック号の生存者予測


　タイタニック号の事故では、2224人の乗客および乗組員のうち1502人が死亡したという。つまり半分以上が死亡したわけだが、性別や年齢、貧富で死亡率にばらつきがあった。<br>
　今回は、どのような人々が生き残る可能性が高いのか分析する。

## 問題
<h3>　すでに生死が判明している891名の個人情報を分析し、判明していない418名の生死について予測せよ。</h3>

つまり言い換えれば

## 問題
<h3>　891個の訓練データの属性を分析し、418個の未知のデータについて二項分類せよ。</h3>

　今回のように生死が判明している乗組員のデータが入手できる場合、**教師あり学習(Supervised learning)**を行うことができる。教師あり学習とは、与えられた**訓練データ(Training data)**をもとに予測を行うモデルを構築する手法のことである。<br>
　また今回は生死のみを予測する問題である。生きてさえいれば容体に関わらずひとくくりに生存者として扱われる。このような2クラスの分類問題をとくに**二項分類(Binary classification)**という。<br>

　訓練データが与えられた分類問題において用いられる機械学習アルゴリズムとして
 
- サポートベクトルマシン
- K近傍法
- アンサンブル分類

などがあげられる。<br>
 
![](http://scikit-learn.org/stable/_static/ml_map.png)

<a href="http://scikit-learn.org/stable/tutorial/machine_learning_map/">機械学習ライブラリscikit-learnが提供するアルゴリズムのチートシート</a>

## 1.1.ライブラリのインポート

　では分析を始めよう。Pythonは機械学習に必要な機能を提供するライブラリが充実しているため、今回はPythonを採用する。

### scikit-learn

　scikit-learnはすでに構築済みの機械学習アルゴリズムを提供するライブラリである。これを用いることで、たとえ数理的な理解をしていなくても機械学習の実装が行える。

In [ ]:
!pip install scikit-learn
#!pip3 install scikit-learn

In [ ]:
import sklearn

### SciPy

　SciPyは高度な科学計算を提供するライブラリである。scikit-learnはこのライブラリを用いて(私たちの代わりに)高度な科学計算を行う。

In [ ]:
!pip install scipy
#!pip3 install scipy

In [ ]:
import scipy as sp

### Pandas

　Pandasはデータ解析ツールを提供するライブラリである。これを用いてデータを取り扱う。

In [ ]:
!pip install pandas
#!pip3 install pandas

In [ ]:
import pandas as pd

 ### Matplotlib
 
 　Matplotlibはグラフ描画ツールを提供するライブラリである。これを用いてグラフを描画する。

In [ ]:
!pip install matplotlib
#!pip3 install matplotlib

In [ ]:
import matplotlib
%matplotlib inline

### Numpy

　Numpyは効率的な数値計算を提供するライブラリである。Pythonで数値計算を行う場合、必ずと言っていいほどNumpyが用いられる。

In [ ]:
!pip install numpy
#!pip3 install numpy

In [ ]:
import numpy as np

## 1.2.データの読み込みと確認

　まずCSVファイルからデータを読み取り、Pandasの提供するデータ構造、DataFrameに変換する。

In [ ]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

　trainの型を確認してみよう。

In [ ]:
type(train)

確かに、CSVファイルから読み取ったデータがDataFrame型に変換されている。<br>
　PandasはDataFrameのほかにSeriesというデータ構造も提供する。Seriesは一次元データのためのデータ構造である。

In [ ]:
type(train['Survived'])

　Pandasによるデータ解析では、多次元データのDataFrameと一次元データのSeriesが繰り返し登場する。

　先ほど読み込んだtrainとtestのDataFrameを確認しよう。<br>
 　headメソッドを用いることで、データ構造の上からn個のデータを表示できる。

In [ ]:
train.head(3)

In [ ]:
test.head(3)

　訓練データであるtrainは生存者Survivedに関する情報が与えられているが、テストデータであるtestには与えられていない。彼らの生存を予測することが今回の分析の目的である。

　infoメソッドを用いることで、DataFrameに関する情報を表示することができる。

In [ ]:
train.info()

In [ ]:
test.info()

　RangeIndexはそのDataFrameの行数を表している。このことからtrainとtestでそれぞれ891人と418人分のデータが存在することがわかる。しかし、例えばAgeのデータは、それぞれ714人と332人分しか存在していない。これはつまり一部データでAgeに関する情報が欠損していることを示している。同様に、CabinやEmbarkedでも欠損が確認できる。

isnullメソッドを用いることで、データ構造の欠損(NaN)について確認できる。

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

## 1.3.データクリーニング

　学習の際に予測と関係のない不要な情報が混じっていると、それに引っ張られて予測の精度を落とすことがある。そこであらかじめ予測に役立ちそうな情報を選んでおく。またデータの欠損も埋めておく。
 
|列名|意味|
|---|---|
|**PassengerId**|ID|
|**	Pclass**|客室のグレード|
|**Name**|名前|
|**Sex**|性別|
|**Age**|年齢|
|**SibSp**|兄弟の数|
|**Parch**|親や子の数|
|**Ticket**|チケット番号|
|**Fare**|乗船料金|
|**Cabin**|部屋番号|
|**Embarked**|乗船した港|

　これらの属性のうち、例えばTicketやCabinは、生存を予測するにあたって必要な情報だろうか。確かにタイタニック号が沈むにあたって、浸水は船の底面から始まっただろうから、下層の客室ほど生存率は低いだろう。しかし部屋番号を表すCabinは欠損が多く、そもそも部屋番号と部屋の位置の対応づけができない(これはTicketについても同じことが言える)。<br>
　しかも部屋の上下関係はPclassやFare反映されているだろうから、やはりTicketやCabinは必要ないだろう。
 
|列名|意味|
|---|---|
|**PassengerId**|ID|
|**	Pclass**|客室のグレード|
|**Name**|名前|
|**Sex**|性別|
|**Age**|年齢|
|**SibSp**|兄弟の数|
|**Parch**|親や子の数|
|**Fare**|乗船料金|
|**Embarked**|乗船した港|

　SibSpやParchについてはどうだろうか。例えば、親は子どもを生かそうとするだろうし、子どもが成人の場合は親を助けるかもしれない。Parchは残しておく価値があるだろう。兄弟についても、お互いに協力したり救出したりする可能性は十分に考えられるから、SibSpも有用な属性に思える。<br>
　そこでSibSpとParchを合計したFamilyという属性を追加する。
 
|列名|意味|
|---|---|
|**PassengerId**|ID|
|**	Pclass**|客室のグレード|
|**Name**|名前|
|**Sex**|性別|
|**Age**|年齢|
|**SibSp**|兄弟の数|
|**Parch**|親や子の数|
|**Fare**|乗船料金|
|**Embarked**|乗船した港|
|**FamilySize**|家族の数|
|**IsAlone**|家族の有無|

　名前は生死の予測に寄与しない。しかし、名前についているMsやMrといった敬称は予測に使えるかもしれない。<br>
　敬称を表す属性を追加しておく。
 
|列名|意味|
|---|---|
|**PassengerId**|ID|
|**	Pclass**|客室のグレード|
|**Name**|名前|
|**Sex**|性別|
|**Age**|年齢|
|**SibSp**|兄弟の数|
|**Parch**|親や子の数|
|**Fare**|乗船料金|
|**Embarked**|乗船した港|
|**FamilySize**|家族の数|
|**IsAlone**|家族の有無|
|**Title**|敬称|

　それでは実際にデータを加工しよう。<br>
 　dropメソッドを用いることで、DataFrameの特定の列または行を削除することができる。

In [ ]:
datasets = [train, test]
columns=['Ticket', 'Cabin']

for dataset in datasets:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 1
    dataset.loc[dataset['FamilySize']  > 1, 'IsAlone'] = 0
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    dataset.drop(columns, axis=1, inplace=True)

In [ ]:
train.head(3)

In [ ]:
test.head(3)

value_countsメソッドを用いることで、要素の個数を表示することができる。

In [ ]:
train['Title'].value_counts()

In [ ]:
test['Title'].value_counts()

　ちなみにvalue_countsメソッドで得られる型は

In [ ]:
type(train['Title'].value_counts())

一次元データなのでSeries型である。またPandasの提供するデータ構造では

In [ ]:
train['Title'].value_counts() < 10

比較演算子を用いて各要素の真理値を出すこともできる。

　DrやRevなど、極端に少ない敬称がいくつか確認できる。これらは外れ値とみなし、Otherという要素として扱うことにする。<br>
　比較演算子を用いて外れ値を探し出し、敬称をOtherに書き換える。

In [ ]:
for dataset in datasets:
    title_names = (dataset['Title'].value_counts() < 10)
    dataset['Title'] = dataset['Title'].apply(lambda x: 'Other' if title_names.loc[x] == True else x)
    dataset.drop('Name', axis=1, inplace=True)

In [ ]:
train['Title'].value_counts()

In [ ]:
test['Title'].value_counts()

　次にAge、EmbarkedそしてFareの欠損の処理を行う。処理方法はさまざまだが、今回はメディアンで欠損を補完する。<br>
　fillnaメソッドを用いることで、データ構造の欠損(NaN)に値を代入できる。

In [ ]:
for dataset in datasets:
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

　最後に、加工したデータをCSVファイルで保存しておく。

In [ ]:
train.to_csv('./processed/train1.csv')
test.to_csv('./processed/test1.csv')